# Building a Spam Filter with Naive Bayes 

In this project, we study the practical side of Naive Bayes algorithm by building a spam filter for SMS messages.

Our first task is to teach the computer how to classify messages. To do this, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset used in this project can be downloaded directly from [this link](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

Let's start by reading in the dataset.

## Read in the data

In [1]:
import pandas as pd
from csv import reader

In [2]:
#datapoints are separated by '/t',the dataset doesn't have a header, assign column labels 
sms_data = pd.read_csv("SMSSpamCollection", sep = "\t", header = None, names = ['Label', 'SMS'])
sms_data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
print(sms_data.shape)
sms_data["Label"].value_counts(dropna=False, normalize = True)*100

(5572, 2)


ham     86.593683
spam    13.406317
Name: Label, dtype: float64

By looking at the frequency distribution of "Label" column in the dataset, we see that about 86.6% of the messages are ham ("ham" means non-spam) and the remaining 13.4% of the messages are spam.  

## Splitting the dataset into 'training set' and 'test set'

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80%.

Hence, once we build our spam filter, we'll need to test how good it is with classifying new messages. To test the spam filter, we are first going to split our dataset into two categories:

- A **training set**, which we'll use to "train" the computer on how to classify messages 
- A **test set**, which we'll use to test how well the spam filter works in classifying new messages


We are going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset)

Let's start by:

- Randomizing the entire dataset to ensure that spam and ham messages are spread properly throughout the dataset.
- Split the randomized dataset into a training and a test set, with the training set accounting for 80% of the dataset and the remaining 20% of the dataset as test set.
- Find the percentage of spam and ham in both the training and the test set to check if they are similar to what we have in the full dataset.



In [4]:
# Use the parameter frac = 1 to randomize the entire dataset
# Use the random_state = 1 parameter to make sure our results are reproducible
randomized_data = sms_data.sample(frac = 1, random_state = 1)
print(randomized_data.shape)
#split the data into training and test set.
training_data = randomized_data[:4458].reset_index(drop = True)
test_data = randomized_data[4458:].reset_index(drop = True)

#checking the percentage of spam and ham in both the training and the test set. 
print(training_data.shape)
print("\n")
print(training_data["Label"].value_counts(normalize = True)*100)
print("\n")
print(test_data.shape)
print("\n")
print(test_data["Label"].value_counts(normalize = True)*100)


(5572, 2)
(4458, 2)


ham     86.54105
spam    13.45895
Name: Label, dtype: float64


(1114, 2)


ham     86.804309
spam    13.195691
Name: Label, dtype: float64


By looking at the frequency distributions of both training dataset and test dataset, we see that the distribution of messages across spam and non-spam in both the datasets is similar to that of the full dataset.

## Data cleaning to compute the probabilities

When a new message comes in, the Naive Bayes algorithm will make the classification based on the computed probabilities. 

To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

To make the calculations easier, we want bring the data to the following format:

|   | Label | secret | prize | claim | now | coming | to | my| party | winner|
| --- | --- | ------ | ------ | ----- | ---- | ---- | ---- | ---- | ----- | ----- |
| 0  | spam  | 2 | 2 | 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1  | ham  | 1 | 0 | 0 | 0 | 1 | 1 | 1 | 1 | 0 |

- The SMS column doesn't exist anymore.
- Instead, the SMS column is replaced by a series of new columns, where each column represents a unique word from the vocabulary.
- Each row describes a single message. 
- For instance,first row corresponds to the message "SECRET PRIZE! CLAIM SECRET PRIZE NOW!!", and it has the values spam, 2, 2, 1, 1, 0, 0, 0, 0, 0. These values tell us that:
 * The message is spam.
 * The word "secret" occurs two times inside the message.
 * The word "prize" occurs two times inside the message.
 * The word "claim" occurs one time inside the message.
 * The word "now" occurs one time inside the message.
 * The words "coming", "to", "my", "party", and "winner" occur zero times inside the message.
- All words in the vocabulary are in lower case, so "SECRET" and "secret" come to be considered to be the same word.
- Punctuation is not taken into account anymore. 

Let's begin the data cleaning process by removing the punctuation and bringing all the words to lower casee.

In [5]:
# Remove the punctuation from the SMS column
# Transform every letter in every word to lower case
pd.set_option('mode.chained_assignment', None)
training_data['SMS'] = training_data['SMS'].str.replace(r'\W', ' ')
training_data['SMS'] = training_data['SMS'].str.lower()

## Cleaned training dataset 

In [6]:
training_data.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


## Creating the vocabulary

Let's create a vocabulary for the messages in the training dataset.  The vocabulary is a Python list containing all the unique words across all messages, where each word is represented as a string. We begin by:

- Transforming each message from the SMS column into a list of words by splitting the string at the space character
- Create a vocabulary list and remove the duplicates

In [7]:
#Transforming each message from the SMS column into a list by splitting the string at the space character
training_data["SMS"] = training_data["SMS"].str.split()

#Initiate an empty list named vocabulary
vocabulary = []

#Appending all words across all messages to the list vocabulary 
for message in training_data["SMS"]:
    for word in message:
        vocabulary.append(word)

#Removing the duplicates from the vocabulary list 
vocabulary = list(set(vocabulary))
print(len(vocabulary))
vocabulary[0:5]

7783


['confused', 'ger', 'edwards', 'depressed', 'key']

## Making the data transformation

We're now going to use the vocabulary to make the data transformation we need. 

Eventually, we're going to create a new DataFrame in the format:

|   | Label | secret | prize | claim | now | coming | to | my| party | winner|
| --- | --- | ------ | ------ | ----- | ---- | ---- | ---- | ---- | ----- | ----- |
| 0  | spam  | 2 | 2 | 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| 1  | ham  | 1 | 0 | 0 | 0 | 1 | 1 | 1 | 1 | 0 |


In [8]:
#Initialize a dictionary named word_counts_per_sms, where each key is a unique word (a string) from the vocabulary
#and each value is a list of the length of training set, where each element in the list is a 0
word_counts_per_sms = {unique_word: [0] * len(training_data['SMS']) 
                       for unique_word in vocabulary}

#Loop over every message in training_data 
#Use enumerate to get both index and message
#Loop over each word in the message and 
#Increment the counter for the word in the dictionary by 1 in the place of its index in the list
for index,message in enumerate(training_data['SMS']):
    for word in message:
        word_counts_per_sms[word][index] += 1

In [9]:
# Converting word_counts_per_sms dictionary into word_counts dataframe
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [10]:
#concatenate the new word_counts dataframe with the training_data dataframe to also add 'Label' and 'SMS' columns 
new_data = pd.concat([word_counts, training_data], axis = 1)
new_data.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥,Label,SMS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ham,"[yep, by, the, pretty, sculpture]"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ham,"[yes, princess, are, you, going, to, make, me,..."
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ham,"[welp, apparently, he, retired]"
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ham,[havent]
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [11]:
new_data.shape

(4458, 7785)

## Creating the spam filter

The Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

The terms P(Spam) and P(Ham), N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub> will have the same value for every new message

- N<sub>Spam</sub> is equal to the total number of words in all the spam messages.
- N<sub>Ham</sub> is equal to the total number of words in all the non-spam messages.
- We'll also use Laplace smoothing and set alpha = 1

In [12]:
#Laplace smoothing
alpha = 1

# Calculating P(Spam) and P(Ham)
p_ham = new_data["Label"].value_counts(normalize=True)['ham']
p_spam = new_data["Label"].value_counts(normalize=True)['spam']
print(p_ham)
print(p_spam)

#Calculating N_vocabulary
n_vocab = len(vocabulary)
print(n_vocab)

0.8654104979811574
0.13458950201884254
7783


In [13]:
#Create a new column 'no_of_words' and assign the number of words in each message to the column 
new_data["no_of_words"] = new_data.iloc[:,:-2].sum(axis = 1)
new_data["no_of_words"].head()

0     5
1     9
2     4
3     1
4    26
Name: no_of_words, dtype: int64

In [14]:
#Calculating N_Spam and N_Ham
n_spam = new_data.loc[new_data['Label'] == 'spam','no_of_words'].sum()
n_ham = new_data.loc[new_data['Label'] == 'ham','no_of_words'].sum()
print("Number of words in all the spam messages is {0}".format(n_spam))
print("Number of words in all the ham messages is {0}".format(n_ham))

Number of words in all the spam messages is 15190
Number of words in all the ham messages is 57237


## Calculating parameters

The parameters P(wi|Spam) and P(wi|Ham) vary depending on the word, but the probability for each individual word is constant for every new message. Let's now calculate all the parameters using the equations below:


\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


In [15]:
#Initialize two dictionaries, where each key-value pair is a unique word (from our vocabulary) represented as a string, and the value is 0. 
#spam_dict dictionary to store the parameters for P(wi|Spam), and ham_dict for P(wi|Ham)
spam_dict = {unique_word: 0 for unique_word in vocabulary}
ham_dict = {unique_word: 0 for unique_word in vocabulary}

#Splitting the cleaned data into spam_data and ham_data 
spam_data = new_data[new_data["Label"] == 'spam']
ham_data = new_data[new_data["Label"] == 'ham']

#calculating the number of times each word occurs across all spam messages
for unique_word in spam_dict:
    spam_dict[unique_word] = spam_data[unique_word].sum()

#calculating the number of times each word occurs across all ham messages
for unique_word in ham_dict:
    ham_dict[unique_word] = ham_data[unique_word].sum()

In [16]:
#calculating the paramters i.e, the conditional probability for each word 
for unique_word in vocabulary:
    #Calculating P(wi|Spam)
    numerator_1 = spam_dict[unique_word] + alpha
    denominator_1 = (n_spam + alpha * n_vocab)
    spam_dict[unique_word] = numerator_1/denominator_1
    
    #Calculating P(wi|Ham)
    numerator_2 = ham_dict[unique_word] + alpha
    denominator_2 = (n_ham + alpha * n_vocab)
    ham_dict[unique_word] = numerator_2/denominator_2


In [18]:
#Converting spam_dict into a dataframe
spam_parameters = pd.DataFrame.from_dict(spam_dict,
                                   orient='index')
spam_parameters = spam_parameters.rename(columns={0: 'spam'})
print(spam_parameters.head())

#Converting ham_dict into a dataframe
ham_parameters = pd.DataFrame.from_dict(ham_dict,
                                   orient='index')
ham_parameters = ham_parameters.rename(columns={0: 'ham'})
print(ham_parameters.head())

#Concatenating two dataframes into parameters dataframe
parameters = pd.concat([spam_parameters, ham_parameters], axis = 1)
parameters.head()

              spam
confused  0.000044
ger       0.000044
edwards   0.000044
offense   0.000044
what      0.000653
               ham
confused  0.000031
ger       0.000031
edwards   0.000031
offense   0.000031
what      0.003476


,spam,ham
confused,0.000044,0.000031
ger,0.000044,0.000031
edwards,0.000044,0.000031
offense,0.000044,0.000031
what,0.000653,0.003476


## Classifying a new message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
- If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
- If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
- If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [25]:
import re

def classify(message):
    
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in parameters.index:
            p_spam_given_message = p_spam_given_message * parameters.loc[word,'spam']
            p_ham_given_message = p_ham_given_message * parameters.loc[word,'ham']
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [26]:
print(classify("WINNER!! This is the secret code to unlock the money: C3421."))

print(classify("Sounds good, Tom, then see u there"))

spam
ham


## Measuring the spam filter's accuracy

In [27]:
#Creating a new column 'predicted' in the test dataset with the value returned by the classify function
test_data['predicted'] = test_data['SMS'].apply(classify)
test_data.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [28]:
total = test_data.shape[0]
mask = (test_data['Label'] == test_data['predicted'])
test_correct = mask.sum()
print(test_correct)
accuracy = (test_correct/total) * 100
print(accuracy)

1100
98.74326750448833


The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.